In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sb
import re
from itertools import combinations
from tqdm.notebook import tqdm
import glob
import datetime

plt.style.use("dark_background")

path = "C:/Windows/Fonts/malgun.ttf"
font_name = mpl.font_manager.FontProperties(fname=path).get_name()
mpl.rc("font", family=font_name)
mpl.rc("axes", unicode_minus=False)

## 협업플랫폼

### Data Preprocessing

In [5]:
target_year = 2021
target_month = 7
pref = "D:/Github/Work/Tableau/Collaboration Tool Usage/"

hr = pd.read_excel(f"D:/Github/Work/Tableau/직원 명단_210731.xlsx")
dooray = pd.read_csv(f"{pref}Raw Data_Collaboration Tool Usage/협업플랫폼_전체_210701-210731.csv")

In [6]:
dooray["post_created_at"] = pd.to_datetime(dooray["post_created_at"])
dooray["comment_created_at"] = pd.to_datetime(dooray["comment_created_at"])

In [8]:
hr = hr.drop_duplicates()
hr = hr.rename({"사번":"num", "성명":"name", "EMAIL":"email", "본부명(조직단위기준)":"hq", "원소속명":"dep", "직급":"rank", "구분":"type"}, axis=1)
hr = hr[["num", "name", "email", "hq", "dep", "rank", "type"]]

# 휴직 상태인 직원은 제외합니다.
# hr = hr[~hr["dep"].str.contains("휴직")]
# 현장의 "hq"는 "현장"으로 변환합니다.
# hr.loc[hr["type"].str.contains("현장"), "hq"] = "현장"
# 신규준비는 해당 팀 소속으로 간주합니다.
# hr["dep"] = hr["dep"].str.replace("(신규준비)", "", regex=False)
# 대기는 해당 팀 소속으로 간주합니다.
# hr["dep"] = hr["dep"].str.replace("(대기)", "", regex=False)
# "(연구실)", "(착공준비)"는 해당 팀 소속으로 간주합니다.
# hr["dep"] = hr["dep"].str.replace("(연구실)", "", regex=False)
# hr["dep"] = hr["dep"].str.replace("(착공준비)", "", regex=False)
# 지사 소속 직원은 제외합니다.
# hr = hr[hr["type"]!="지사"]
# hr.loc[hr["dep"].str.contains("건축PAM"), "dep"] = "건축PAM"
# hr.loc[hr["dep"].str.contains("인프라PAM"), "dep"] = "인프라PAM"

# hr.loc[hr["num"]==1101290, "hq"] = "개발영업본부"

email2name = {email:name for email, name in zip(hr["email"], hr["name"])}
email2dep = {email:dep for email, dep in zip(hr["email"], hr["dep"])}
email2hq = {email:hq for email, hq in zip(hr["email"], hr["hq"])}

### 최근 부서별 사용량.csv

In [60]:
data[(data["post_from"] == "6363@hdc-dvp.com") & (data["post_created_at"].dt.year == target_year) & (data["post_created_at"].dt.month == target_month)].drop_duplicates(["post_id"])

,project_id,project_name,post_id,post_subject_size,post_body_size,post_from,post_to,post_cc,post_created_at,comment_id,comment_body_size,commenter,comment_created_at
279405,2590735541878145948,IT-신청,3038300713542430032,36,483,6363@hdc-dvp.com,['4100422@hdc-dvp.com'],"['wkcho@hdc-dvp.com','mej@hdc-dvp.com','han96m...",2021-06-24,3.038311e+18,19.0,wkcho@hdc-dvp.com,2021-06-24
280773,2597671184417881648,Go-혁신,3023891813996122407,50,614,6363@hdc-dvp.com,['6363@hdc-dvp.com'],"['5969@hdc-dvp.com','mej@hdc-dvp.com','6445@hd...",2021-06-04,3.026161e+18,224.0,6363@hdc-dvp.com,2021-06-07
280800,2597671184417881648,Go-혁신,3032593156325597879,47,1081,6363@hdc-dvp.com,['6363@hdc-dvp.com'],"['6299@hdc-dvp.com','kyungkeun.yoo@hdc-dvp.com...",2021-06-16,NaN,NaN,NaN,NaT
293351,2981038588610011468,미래혁신본부-보고,3042887262052256600,56,916,6363@hdc-dvp.com,"['kyungkeun.yoo@hdc-dvp.com','6363@hdc-dvp.com']","['mej@hdc-dvp.com','4001341@hdc-dvp.com','hikl...",2021-06-30,NaN,NaN,NaN,NaT
293411,2981892333338475758,고객관리혁신-HelpDesk,3027404296285317447,73,795,6363@hdc-dvp.com,['6363@hdc-dvp.com'],"['6048@hdc-dvp.com','hyejin.kim@hdc-dvp.com','...",2021-06-09,3.028366e+18,648.0,4100422@hdc-dvp.com,2021-06-10


In [ ]:
# 최근 1개월 간 작성된 업무만 뽑습니다.
recent_posts = data[(data["post_created_at"].dt.year == target_year) & (data["post_created_at"].dt.month == target_month)]
# 타사 직원이 작성한 업무는 제거합니다.
# recent_posts = recent_posts[recent_posts["post_from"] != "0"]
# 중복된 업무는 제거합니다.
recent_posts = recent_posts.drop_duplicates(["post_id"])
recent_posts = pd.merge(recent_posts, hr, left_on="post_from", right_on="email", how="outer")
recent_posts = recent_posts[["project_name", "hq", "dep", "name", "post_body_size"]]
recent_post = recent_posts.groupby(["hq", "dep", "name"]).count()["post_body_size"]
recent_post_size = recent_posts.groupby(["hq", "dep", "name"]).sum()["post_body_size"]

# 최근 1개월 간 작성된 댓글만 뽑습니다.
recent_comments = data[(data["comment_created_at"].dt.year==target_year) & (data["comment_created_at"].dt.month==target_month)]
# 타사 직원이 작성한 댓글은 제거합니다.
# recent_comments = recent_comments[recent_comments["commenter"] != "0"]
recent_comments = pd.merge(recent_comments, hr, left_on="commenter", right_on="email", how="outer")
recent_comments = recent_comments[["project_name", "hq", "dep", "name", "comment_body_size"]]
recent_comment = recent_comments.groupby(["hq", "dep", "name"]).count()["comment_body_size"]
recent_comment_size = recent_comments.groupby(["hq", "dep", "name"]).sum()["comment_body_size"]

temp = pd.merge(recent_post, recent_comment, left_index=True, right_index=True, how="outer")
temp = pd.merge(temp, recent_post_size, left_index=True, right_index=True)
merge = pd.merge(temp, recent_comment_size, left_index=True, right_index=True)
merge.columns = ["post", "comment", "post_size", "comment_size"]

hr_temp = hr[~hr["hq"].str.contains("회장직속|부회장직속")]
idx = pd.MultiIndex.from_tuples(list(hr_temp.groupby(["hq", "dep", "name"]).groups))
recent = pd.DataFrame(data=0, index=idx, columns=["post", "comment", "post_size", "comment_size"])

for _, row in merge.iterrows():
    recent.loc[row.name] = row
    
recent = recent.reset_index(drop=False).rename({"level_0":"hq", "level_1":"dep", "level_2":"name"}, axis=1)
recent = recent[~recent["dep"].str.contains("회장|사장|본부|상임자문|비상근")]

In [6]:
# recent.to_csv(f"{pref}최근 부서별 사용량.csv", encoding="euc-kr", index=False)
recent.head()

,hq,dep,name,post,comment,post_size,comment_size
0,개발영업본부,H1프로젝트,김 현석,0.0,0.0,0.0,0.0
1,개발영업본부,H1프로젝트,김 호중,0.0,7.0,0.0,126.0
2,개발영업본부,H1프로젝트,박 주원,1.0,4.0,975.0,713.0
3,개발영업본부,H1프로젝트,윤 기락,0.0,0.0,0.0,0.0
4,개발영업본부,H1프로젝트,임 도훈,0.0,1.0,0.0,53.0


### 보고방별 보고 건수.csv

In [28]:
prj2head = {"회장-보고":hr.loc[hr["dep"]=="회장", "email"].item(), "부회장-보고":hr.loc[hr["dep"]=="부회장", "email"].item(), "대표이사사장-보고":hr.loc[hr["dep"]=="대표이사사장", "email"].item(), "개발영업본부-보고":hr.loc[hr["dep"]=="개발영업본부", "email"].item(), "건설본부-보고":hr.loc[hr["dep"]=="건설본부", "email"].item(), "경영본부-보고":hr.loc[hr["dep"]=="경영본부", "email"].item(), "미래혁신본부-보고":hr.loc[hr["dep"]=="미래혁신본부", "email"].item()}
head2prj = {value:key for key, value in prj2head.items()}

#  & (dooray["post_from"] == "6363@hdc-dvp.com")
reports = dooray[(dooray["project_name"].isin(prj2head)) & (dooray["commenter"].isin(head2prj))].drop_duplicates(["post_id", "commenter"])

reports = pd.merge(reports, hr[["email", "hq", "dep", "name"]], left_on="post_from", right_on="email").groupby(["project_name", "hq", "dep", "name"], as_index=False).size()

In [29]:
# reports.to_csv(f"{pref}reports.csv", index=False, encoding="euc-kr")
reports.head()

,project_name,hq,dep,name,size
0,개발영업본부-보고,개발영업본부,민간수주1팀,한 상현,1
1,개발영업본부-보고,개발영업본부,용산병원프로젝트,진 보은,1
2,개발영업본부-보고,개발영업본부,인프라개발팀,김 대호,1
3,개발영업본부-보고,개발영업본부,인프라수주팀,김 영현,1
4,개발영업본부-보고,개발영업본부,인프라수주팀,배 종민,1


### 연월별 사용량.csv

In [9]:
data_posts = data.drop_duplicates(["post_id"])
post_month = data_posts.groupby(pd.Grouper(key="post_created_at", freq="M")).size()
post_month.name = "post"
post_size_month = data_posts.groupby(pd.Grouper(key="post_created_at", freq="M")).sum()["post_body_size"]
post_size_month.name = "post_size"

comment_month = data.groupby(pd.Grouper(key="comment_created_at", freq="M")).size()
comment_month.name = "comment"
comment_size_month = data.groupby(pd.Grouper(key="post_created_at", freq="M")).sum()["comment_body_size"]
comment_size_month.name = "comment_size"

temp = pd.merge(post_month, comment_month, left_index=True, right_index=True, how="outer")
temp = pd.merge(temp, post_size_month, left_index=True, right_index=True, how="inner")
month = pd.merge(temp, comment_size_month, left_index=True, right_index=True, how="inner")

In [10]:
month.to_csv(f"{pref}연월별 사용량.csv", encoding="euc-kr")
month.tail()

,post,comment,post_size,comment_size
2021-02-28,8031.0,10315,58007365.0,1567083
2021-03-31,9949.0,9938,78496434.0,1499882
2021-04-30,9590.0,9212,74380327.0,1948546
2021-05-31,8954.0,9945,65439333.0,2121287
2021-06-30,10499.0,12357,73883408.0,1585326


### 연월별 보고 건수.csv

In [40]:
reports2 = dooray[(dooray["project_name"].isin(prj2head)) & (dooray["commenter"].isin(head2prj))].drop_duplicates(["post_id"])

In [41]:
reports2

,project_id,project_name,post_id,post_subject_size,post_body_size,post_from,post_to,post_cc,post_created_at,comment_id,comment_body_size,commenter,comment_created_at
3660,2591135457748217935,부회장-보고,3049141000938507956,25,1024,ymsm@hdc-dvp.com,"['6186@hdc-dvp.com','ymsm@hdc-dvp.com']","['beauiful@hdc-dvp.com','kmhanjs@hdc-dvp.com',...",2021-07-09,3.049287e+18,16.0,dckim@hdc-dvp.com,2021-07-09
3665,2591135457748217935,부회장-보고,3053640608265470967,29,872,6186@hdc-dvp.com,['jhsjung@hdc-dvp.com'],"['beauiful@hdc-dvp.com','kmhanjs@hdc-dvp.com',...",2021-07-15,3.053656e+18,129.0,dckim@hdc-dvp.com,2021-07-15
3666,2591135457748217935,부회장-보고,3063775805246081336,29,405,youngjoo@hdc-dvp.com,['youngjoo@hdc-dvp.com'],"['beauiful@hdc-dvp.com','dckim@hdc-dvp.com','k...",2021-07-29,3.063802e+18,245.0,dckim@hdc-dvp.com,2021-07-29
3675,2591135918625451217,개발영업본부-보고,3049435353675627275,60,1248,5952@hdc-dvp.com,['5952@hdc-dvp.com'],"['park1@hdc-dvp.com','eskim@hdc-dvp.com','si53...",2021-07-09,3.049477e+18,16.0,park1@hdc-dvp.com,2021-07-09
3693,2591135918625451217,개발영업본부-보고,3059261912334376146,82,337,hansh77@hdc-dvp.com,"['hansh77@hdc-dvp.com','okiy2400@hdc-dvp.com']","['park1@hdc-dvp.com','suna.yang@hdc-dvp.com']",2021-07-23,3.059278e+18,18.0,park1@hdc-dvp.com,2021-07-23
...,...,...,...,...,...,...,...,...,...,...,...,...,...
22329,2981038588610011468,미래혁신본부-보고,3038991977186351793,43,1108,han96mir@hdc-dvp.com,['han96mir@hdc-dvp.com'],"['mej@hdc-dvp.com','4001341@hdc-dvp.com','hikl...",2021-06-25,3.043281e+18,84.0,hikli@hdc-dvp.com,2021-07-01
22332,2981038588610011468,미래혁신본부-보고,3046412656022965409,26,676,ymsm@hdc-dvp.com,"['6186@hdc-dvp.com','ymsm@hdc-dvp.com']","['koyh@hdc-dvp.com','4001341@hdc-dvp.com','hik...",2021-07-05,3.047719e+18,459.0,hikli@hdc-dvp.com,2021-07-07
22340,2981038588610011468,미래혁신본부-보고,3049313544095290467,78,2920,han96mir@hdc-dvp.com,"['6090@hdc-dvp.com','han96mir@hdc-dvp.com']","['mej@hdc-dvp.com','wh@hdc-dvp.com','hikli@hdc...",2021-07-09,3.052977e+18,164.0,hikli@hdc-dvp.com,2021-07-14
22349,2981038588610011468,미래혁신본부-보고,3052972031984247375,33,1787,6090@hdc-dvp.com,['6090@hdc-dvp.com'],"['mej@hdc-dvp.com','wh@hdc-dvp.com','4001341@h...",2021-07-14,3.052989e+18,123.0,hikli@hdc-dvp.com,2021-07-14


In [11]:
bogo = data[data["project_name"].isin(prj2head.keys())]
bogo = bogo.drop_duplicates(["post_id"])
bogo_month = bogo.groupby(pd.Grouper(key="post_created_at", freq="M")).size()

In [12]:
bogo_month.to_csv(f"{pref}연월별 보고 건수.csv", encoding="euc-kr")
bogo_month.tail()

post_created_at
2021-02-28     24
2021-03-31     35
2021-04-30     35
2021-05-31    128
2021-06-30     82
Freq: M, dtype: int64

# TEST

In [53]:
reports[reports["post_from"] == "6363@hdc-dvp.com"]

,project_id,project_name,post_id,post_subject_size,post_body_size,post_from,post_to,post_cc,post_created_at,comment_id,comment_body_size,commenter,comment_created_at
3749,2591136577788543151,대표이사사장-보고,3042887262052256600,56,916,6363@hdc-dvp.com,"['kyungkeun.yoo@hdc-dvp.com','6363@hdc-dvp.com']","['kshjh@hdc-dvp.com','jiniq@hdc-dvp.com','5969...",2021-06-30,3.043367e+18,339.0,hikli@hdc-dvp.com,2021-07-01


In [45]:
dooray["post_id"].tolist()

[3042968026433952821,
 3042968201781709664,
 3047946944018913225,
 3047946944018913225,
 3047946944018913225,
 3047946944018913225,
 3047946944018913225,
 3047946944018913225,
 3047946944018913225,
 3047946944018913225,
 3047946944018913225,
 3047946944018913225,
 3047946944018913225,
 3047946944018913225,
 3047946944018913225,
 3047946944018913225,
 3047946944018913225,
 3047946944018913225,
 3047946944018913225,
 3047946944018913225,
 3047946944018913225,
 3047946944018913225,
 3047946944018913225,
 3047946944018913225,
 3047946944018913225,
 3047946944018913225,
 3047946944018913225,
 3047946944018913225,
 3047946944018913225,
 3048644353159713925,
 3048644353159713925,
 3048644353159713925,
 3048644353159713925,
 3048644353159713925,
 3048644353159713925,
 3048644353159713925,
 3048644353159713925,
 3048644353159713925,
 3048644353159713925,
 3048644353159713925,
 3048644353159713925,
 3048644353159713925,
 3048644353159713925,
 3048644353159713925,
 3048644353159713925,
 304864435

In [54]:
dooray[dooray["post_id"] == 3042887262052256600]["post_cc"].tolist()

["['kshjh@hdc-dvp.com','jiniq@hdc-dvp.com','5969@hdc-dvp.com','mej@hdc-dvp.com','6445@hdc-dvp.com','4001341@hdc-dvp.com','hikli@hdc-dvp.com']",
 "['kshjh@hdc-dvp.com','jiniq@hdc-dvp.com','5969@hdc-dvp.com','mej@hdc-dvp.com','6445@hdc-dvp.com','4001341@hdc-dvp.com','hikli@hdc-dvp.com']",
 "['kshjh@hdc-dvp.com','jiniq@hdc-dvp.com','5969@hdc-dvp.com','mej@hdc-dvp.com','6445@hdc-dvp.com','4001341@hdc-dvp.com','hikli@hdc-dvp.com']",
 "['kshjh@hdc-dvp.com','jiniq@hdc-dvp.com','5969@hdc-dvp.com','mej@hdc-dvp.com','6445@hdc-dvp.com','4001341@hdc-dvp.com','hikli@hdc-dvp.com']",
 "['kshjh@hdc-dvp.com','jiniq@hdc-dvp.com','5969@hdc-dvp.com','mej@hdc-dvp.com','6445@hdc-dvp.com','4001341@hdc-dvp.com','hikli@hdc-dvp.com']",
 "['kshjh@hdc-dvp.com','jiniq@hdc-dvp.com','5969@hdc-dvp.com','mej@hdc-dvp.com','6445@hdc-dvp.com','4001341@hdc-dvp.com','hikli@hdc-dvp.com']"]